In [2]:
import MeCab
txt = "これはテストではありませんでした"
m = MeCab.Tagger("-Ochasen")
print(m.parse(txt))
m2 = MeCab.Tagger("mecabrc")
print(m2.parse(txt))
m3 = MeCab.Tagger("mecab-ipadic-neologd")
print(m3.parse(txt))

これ	コレ	これ	名詞-代名詞-一般		
は	ハ	は	助詞-係助詞		
テスト	テスト	テスト	名詞-サ変接続		
で	デ	で	助詞-格助詞-一般		
は	ハ	は	助詞-係助詞		
あり	アリ	ある	動詞-自立	五段・ラ行	連用形
ませ	マセ	ます	助動詞	特殊・マス	未然形
ん	ン	ん	助動詞	不変化型	基本形
でし	デシ	です	助動詞	特殊・デス	連用形
た	タ	た	助動詞	特殊・タ	基本形
EOS

これ	名詞,代名詞,一般,*,*,*,これ,コレ,コレ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
テスト	名詞,サ変接続,*,*,*,*,テスト,テスト,テスト
で	助詞,格助詞,一般,*,*,*,で,デ,デ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
あり	動詞,自立,*,*,五段・ラ行,連用形,ある,アリ,アリ
ませ	助動詞,*,*,*,特殊・マス,未然形,ます,マセ,マセ
ん	助動詞,*,*,*,不変化型,基本形,ん,ン,ン
でし	助動詞,*,*,*,特殊・デス,連用形,です,デシ,デシ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
EOS

これ	名詞,代名詞,一般,*,*,*,これ,コレ,コレ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
テスト	名詞,サ変接続,*,*,*,*,テスト,テスト,テスト
で	助詞,格助詞,一般,*,*,*,で,デ,デ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
あり	動詞,自立,*,*,五段・ラ行,連用形,ある,アリ,アリ
ませ	助動詞,*,*,*,特殊・マス,未然形,ます,マセ,マセ
ん	助動詞,*,*,*,不変化型,基本形,ん,ン,ン
でし	助動詞,*,*,*,特殊・デス,連用形,です,デシ,デシ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
EOS



In [31]:
def mecab(txt):
    m = MeCab.Tagger("mecab-ipadic-neologd")
    line = m3.parse(txt).split('\n')[:-2]
    words = [l.split('\t')[1].split(',') for l in line]
    return words

imecab("これはテストではありませんでした")

[['名詞', '代名詞', '一般', '*', '*', '*', 'これ', 'コレ', 'コレ'],
 ['助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ'],
 ['名詞', 'サ変接続', '*', '*', '*', '*', 'テスト', 'テスト', 'テスト'],
 ['助詞', '格助詞', '一般', '*', '*', '*', 'で', 'デ', 'デ'],
 ['助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ'],
 ['動詞', '自立', '*', '*', '五段・ラ行', '連用形', 'ある', 'アリ', 'アリ'],
 ['助動詞', '*', '*', '*', '特殊・マス', '未然形', 'ます', 'マセ', 'マセ'],
 ['助動詞', '*', '*', '*', '不変化型', '基本形', 'ん', 'ン', 'ン'],
 ['助動詞', '*', '*', '*', '特殊・デス', '連用形', 'です', 'デシ', 'デシ'],
 ['助動詞', '*', '*', '*', '特殊・タ', '基本形', 'た', 'タ', 'タ']]

In [46]:
words = mecab("照明はボタン１で、エアコンはボタン２で操作します")
# filter noun and verb
[m for m in words if m[0] in ['名詞', '動詞']]

[['名詞', 'サ変接続', '*', '*', '*', '*', '照明', 'ショウメイ', 'ショーメイ'],
 ['名詞', '一般', '*', '*', '*', '*', 'ボタン', 'ボタン', 'ボタン'],
 ['名詞', '数', '*', '*', '*', '*', '１', 'イチ', 'イチ'],
 ['名詞', '一般', '*', '*', '*', '*', 'エアコン', 'エアコン', 'エアコン'],
 ['名詞', '一般', '*', '*', '*', '*', 'ボタン', 'ボタン', 'ボタン'],
 ['名詞', '数', '*', '*', '*', '*', '２', 'ニ', 'ニ'],
 ['名詞', 'サ変接続', '*', '*', '*', '*', '操作', 'ソウサ', 'ソーサ'],
 ['動詞', '自立', '*', '*', 'サ変・スル', '連用形', 'する', 'シ', 'シ']]

In [47]:
import itertools
wor
uniq = [(g[0], len(list(g[1]))) for g in itertools.groupby(words.sort())]

TypeError: 'NoneType' object is not iterable